In [3]:
#from pydataset import data
import pandas as pd 
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [7]:
df = pd.read_csv('df_worlds.csv')
df['name'] = df['name'].astype(str).str.replace(r"\s*\(.*?\)", "", regex=True).str.strip()

ranking_wca = {
    'Yiheng Wang': 1,
    'Xuanyi Geng': 2,
    'Tymon Kolasiński': 3,
    'Ruihang Xu': 4,
    'Max Park': 5,
    'Teodor Zajder': 6,
    'Luke Garrett': 7,
    'Bofan Zhang': 8,
    'Matty Hiroto Inaba': 9
}
df['ranking_wca'] = df['name'].map(ranking_wca)
df['en_podio'] = (df['pos'] <= 3).astype(int)
df.head()

,pos,name,event_id,wca_id,average_seconds,ranking_wca,en_podio
0,2,Bofan Zhang,333,2021ZHAN01,5.62,8,1
1,4,Bofan Zhang,333,2021ZHAN01,5.78,8,0
2,3,Bofan Zhang,333,2021ZHAN01,5.56,8,1
3,2,Bofan Zhang,333,2021ZHAN01,5.72,8,1
4,2,Bofan Zhang,333,2021ZHAN01,6.06,8,1


In [8]:
label_encoder = LabelEncoder()
df['name'] = label_encoder.fit_transform(df['name'])

In [9]:
df['en_podio_prob'] = df.groupby('name')['en_podio'].transform('mean')


In [13]:
X = df[['average_seconds', 'name']]
y = df['en_podio_prob']

df

,pos,name,event_id,wca_id,average_seconds,ranking_wca,en_podio,en_podio_prob
0,2,0,333,2021ZHAN01,5.62,8,1,0.519231
1,4,0,333,2021ZHAN01,5.78,8,0,0.519231
2,3,0,333,2021ZHAN01,5.56,8,1,0.519231
3,2,0,333,2021ZHAN01,5.72,8,1,0.519231
4,2,0,333,2021ZHAN01,6.06,8,1,0.519231
...,...,...,...,...,...,...,...,...
747,1,8,333,2019WANY36,4.44,1,1,0.977099
748,2,8,333,2019WANY36,5.25,1,1,0.977099
749,1,8,333,2019WANY36,3.98,1,1,0.977099
750,1,8,333,2019WANY36,4.40,1,1,0.977099


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)#distribucion de rango 80-20

In [15]:
clf = GaussianNB()
clf.fit(X_train, y_train)

ValueError: Unknown label type: (array([0.51923077, 0.64516129, 0.80769231, 0.88235294, 0.93902439,
       0.97709924, 0.98305085, 0.98630137, 1.        ]),)

In [8]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.60      0.83      0.70       130
           2       0.25      0.05      0.08        40
           3       0.00      0.00      0.00        27
           4       0.00      0.00      0.00        13
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         1
          27       0.00      0.00      0.00         1

    accuracy                           0.49       226
   macro avg       0.08      0.08      0.07       226
weighted avg       0.39      0.49      0.42       226



c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

In [10]:
clf.score(X_train, y_train)

0.5228136882129277

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'var_smoothing': [0.00000001, 0.000000001, 0.0000000001 ]
}

In [12]:
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs = -1)
grid_search.fit(X_train, y_train)

c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-08, 1e-09, 1e-10]},
             scoring='accuracy')

In [13]:
grid_search.best_params_

{'var_smoothing': 1e-08}

In [14]:
grid_search.best_score_

np.float64(0.5569272237196765)